In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import resource
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
# ------- Define modular methods for the task
def log_max_mem_usage():
    print(
        "Current all-time max memory: {} MB".format(
            resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1000
        )
    )

sample_submission.csv
sample_submission.csv.zip
test.csv
test.csv.zip
train.csv
train.csv.zip



In [2]:
train_df = pd.read_csv('../input/train.csv')
train_df.dropna(inplace=True)  # For id: qid2 174364

train_df.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
log_max_mem_usage()

Current all-time max memory: 207 MB


In [4]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion

# featurizers = [
#     ('char_tfidf', TfidfVectorizer(analyzer='char', ngram_range=(2, 3))),
#     ('word_tfidf', TfidfVectorizer(analyzer='word', ngram_range=(1, 2)))
# ]

# char_weight = 0.4
# combined_featurizers = FeatureUnion(
#     featurizers,
#     n_jobs=7,
#     transformer_weights={
#         'char_tfidf': char_weight,
#         'word_tfidf': 1 - char_weight
#     }
# )

unique_questions = pd.Series(pd.concat([train_df.question1, train_df.question2]).unique())
# combined_featurizers.fit(unique_questions)

char_tfidf = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))  # featurizers[0][1]
word_tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2))  # featurizers[1][1]
char_tfidf.fit(unique_questions)
word_tfidf.fit(unique_questions)

log_max_mem_usage()

Current all-time max memory: 2431 MB
CPU times: user 1min 59s, sys: 1.52 s, total: 2min
Wall time: 2min 2s


In [159]:
c = word_tfidf.transform(samp.question1)
c.todense()

matrix([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [226]:
%%time

def get_tfidf_features(data_df, batch=1000):
    i = 0

    word_dataset = np.array([])
    char_dataset = np.array([])

    while True:
        samp = data_df[i * batch: (i + 1) * batch]
        i += 1
        if i * batch % 10000 == 0:
            print(i * batch)

        if samp.empty:
            break

        word_res = np.dot(word_tfidf.transform(samp.question1), word_tfidf.transform(samp.question2).T).diagonal()
        char_res = np.dot(char_tfidf.transform(samp.question1), char_tfidf.transform(samp.question2).T).diagonal()

        word_dataset = np.concatenate([word_dataset, word_res])
        char_dataset = np.concatenate([char_dataset, char_res])

    return pd.DataFrame(dict(wv=word_dataset, cv=char_dataset), index=data_df.index)

log_max_mem_usage()

Current all-time max memory: 2586 MB
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 247 µs


In [234]:
s = 'hel12312sdf565'
s.count('1')

2

In [264]:
from nltk.corpus import stopwords
from collections import Counter
import re


stops = set(stopwords.words("english"))
num_pattern = re.compile('[0-9]+')
nums = '01234567890'

def get_heuristic_scores(q1, q2):
    n_q1 = {}
    n_q2 = {}

    for n in nums:
        qc1 = q1.count(n)
        qc2 = q2.count(n)
        n_q1['q1_{}'.format(n)] = qc1
        n_q2['q2_{}'.format(n)] = qc2

    exact_nums_q1 = num_pattern.findall(q1)
    exact_nums_q2 = num_pattern.findall(q2)
    
    num_exact_nums_match = len([n1 for n1 in exact_nums_q1 if n1 in exact_nums_q2])

#     qq2 = pd.Series(Counter([s for s in q1 if s.isupper()]))
#     qq1 = pd.Series(Counter([s for s in q2 if s.isupper()]))
    
#     sim_caps_rate = (qq1/qq2).mean()
#     num_caps_q1 = qq1.sum() 
#     num_caps_q2 = qq2.sum()

#     mean_caps_q1 = qq1.mean() 
#     mean_caps_q2 = qq2.mean()
    
    num_terms_q1 = len(q1.split())
    num_terms_q2 = len(q2.split())
    
    len_q1 = len(q1)
    len_q2 = len(q2)

    res = dict(
        num_exact_nums_match=num_exact_nums_match,
        len_diff=abs(len_q1 - len_q2),
        len_q1=len_q1,
        len_q2=len_q2,
        word_num_diff=abs(num_terms_q1 - num_terms_q2),
        num_terms_q1=num_terms_q1,
        num_terms_q2=num_terms_q2,
#         sim_caps_rate=sim_caps_rate,
#         mean_caps_q1=mean_caps_q1,
#         mean_caps_q2=mean_caps_q2,
#         num_caps_q1=num_caps_q1,
#         num_caps_q2=num_caps_q2,
    )
    
    res.update(n_q1)
    res.update(n_q2)
    
    return res

def score_row(row):
    _, row = row


    q1 = row.question1
    q2 = row.question2

    if not all([set(q1.lower().split()).difference(stops), set(q2.lower().split()).difference(stops)]):
        print('here!')
        return 0

    return get_heuristic_scores(q1, q2)

In [265]:
%%time
ds = []
samp = train_df[404000:]
for row in samp.iterrows():
    ds.append(score_row(row))

pd.DataFrame(ds, index=samp.index)

CPU times: user 96 ms, sys: 16 ms, total: 112 ms
Wall time: 99.8 ms


In [266]:
%%time
from sklearn.model_selection import train_test_split
import multiprocessing as mp
import time

samp = train_df.head(10000)

def get_features(samp):
    start_time = time.time()
    
    tfidf_features = get_tfidf_features(samp)
    print('Finished computing tfidf features after {} seconds.'.format((time.time() - start_time)))

    heuristics_scores = []

    for row in samp.iterrows():
        heuristics_scores.append(score_row(row))
    
    heuristics_scores = pd.DataFrame(heuristics_scores, index=samp.index)
    
    features = pd.concat([tfidf_features, heuristics_scores], axis=1)
    
    return features

features = get_features(samp)

X_train, X_test, y_train, y_test = train_test_split(features, samp.is_duplicate, test_size=0.7)
log_max_mem_usage()

10000
Finished computing tfidf features after 5.34293198586 seconds.
Current all-time max memory: 2625 MB
CPU times: user 7.51 s, sys: 112 ms, total: 7.62 s
Wall time: 7.54 s


In [267]:
features

,cv,wv,len_diff,len_q1,len_q2,num_exact_nums_match,num_terms_q1,num_terms_q2,q1_0,q1_1,...,q2_1,q2_2,q2_3,q2_4,q2_5,q2_6,q2_7,q2_8,q2_9,word_num_diff
0,0.944067,0.953420,9,66,57,0,14,12,0,0,...,0,0,0,0,0,0,0,0,0,2
1,0.823531,0.561796,37,51,88,0,8,13,0,0,...,0,0,0,0,0,0,0,0,0,5
2,0.398727,0.082986,14,73,59,0,14,10,0,0,...,0,0,0,0,0,0,0,0,0,4
3,0.011027,0.000000,15,50,65,0,11,9,0,0,...,0,4,2,2,0,0,0,0,0,2
4,0.220276,0.077710,37,76,39,0,13,7,0,0,...,0,0,0,0,0,0,0,0,0,6
5,0.489333,0.340561,4,86,90,0,16,16,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0.024651,0.000000,43,19,62,0,4,11,0,0,...,0,0,0,0,0,0,0,0,0,7
7,0.623099,0.289969,11,30,41,0,7,9,0,0,...,0,0,0,0,0,0,0,0,0,2
8,0.256409,0.904062,2,35,37,0,8,8,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0.680133,0.225348,11,60,49,1,9,9,2,0,...,0,0,1,1,0,0,0,0,0,0


In [9]:
%%time
from sklearn.model_selection import train_test_split
import multiprocessing as mp

samp = train_df

def get_features(samp):
    scores_data = []
    
    cpus = mp.cpu_count()
    print('Total cpus: {}'.format(cpus))

    pool = mp.Pool(cpus)
    results = []

    for ix, row in enumerate(samp.iterrows()):
        results.append(pool.apply_async(score_row, (row,)))

    for ix, row_score in enumerate(results):
        row_score = row_score.get()

        if ix % 1000 == 0:
            print(ix)

        scores_data.append(row_score)

    features = pd.DataFrame(scores_data)
    
    return features

features = get_features(samp)

X_train, X_test, y_train, y_test = train_test_split(features, samp.is_duplicate, test_size=0.7)
log_max_mem_usage()

Total cpus: 4
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000


AttributeError: 'int' object has no attribute 'keys'

In [129]:
# http://stackoverflow.com/questions/20727375/multiprocessing-pool-slower-than-just-using-ordinary-functions
import multiprocessing as mp


def do_something(data):
    return data * 2

def consumer(inQ, outQ):
    while True:
        try:
            # get a new message
            val = inQ.get()

            # this is the 'TERM' signal
            if val is None:
                break;

            # unpack the message
            pos = val[0]  # its helpful to pass in/out the pos in the array
            data = val[1]
            x = tuple(data)

            # process the data
            # print(data)
            ret = score_row(data)
            # print(pos, ret)

            # send the response / results
            outQ.put( (pos, ret) )


        except Exception, e:
            if "'str' object has no attribute" in e:
                print "error!", e
                # outQ.put((None, None))
                continue
            else:
                print "error!", e
                # print pos, data
                break

def process_data(data_list, inQ, outQ, n_workers):
    # send pos/data to workers
    for i, dat in enumerate(data_list):
        inQ.put((i, dat))

    # process results
    for i in range(len(data_list)):
        if i % 1000 == 0:
            print(i)
            
        ret = outQ.get()
        pos = ret[0]
        dat = ret[1]
        data_list[pos] = ret

        
def main(samp, batch_size=1000):
    # initialize things
    n_workers = mp.cpu_count()

    inQ = mp.Queue()
    outQ = mp.Queue()

    # instantiate workers
    workers = [mp.Process(target=consumer, args=(inQ,outQ))
               for i in range(n_workers)]

    # start the workers
    for w in workers:
        w.start()

    dataset = []
    data_list = []
    
#     for i, row in enumerate(samp.iterrows()):
#         if i and i % batch_size == 0:
            
#             # gather some data
#             data_list = [row for row in samp.iterrows()]

#             # lets process the data a few times
#             for i in range(4):
#                 process_data(data_list, inQ, outQ)

#             # tell all workers, no more data (one msg for each)
#             for i in range(n_workers):
#                 inQ.put(None)
#             # join on the workers
#             for w in workers:
#                 w.join()

#             dataset.extend(data_list)
# #             # print out final results  (i*16)
# #             for i,dat in enumerate(data_list):
# #                 print i, dat
#             data_list = []

#         data_list.append(row)
    

    # gather some data
    data_list = [row for row in samp.iterrows()]

    # lets process the data a few times
    process_data(data_list, inQ, outQ, n_workers)

    # tell all workers, no more data (one msg for each)
    for i in range(n_workers):
        inQ.put(None)
        
    # join on the workers
    for w in workers:
        w.join()

#     # print out final results  (i*16)
    for i,dat in enumerate(data_list):
        dataset.append(dat)
        
    return pd.DataFrame(dict(dataset)).T

In [130]:
# # http://stackoverflow.com/questions/20727375/multiprocessing-pool-slower-than-just-using-ordinary-functions
# import multiprocessing as mp


# def do_something(data):
#     return data * 2

# def consumer(inQ, outQ):
#     while True:
#         try:
#             # get a new message
#             val = inQ.get()

#             # this is the 'TERM' signal
#             if val is None:
#                 break;

#             # unpack the message
#             pos = val[0]  # its helpful to pass in/out the pos in the array
#             data = val[1]
#             x = tuple(data)

#             # process the data
#             # print(data)
#             ret = score_row(data)
#             # print(pos, ret)

#             # send the response / results
#             outQ.put( (pos, ret) )


#         except Exception, e:
#             if "'str' object has no attribute" in e:
#                 print "error!", e
#                 # outQ.put((None, None))
#                 continue
#             else:
#                 print "error!", e
#                 # print pos, data
#                 break

# def process_data(data_list, inQ, outQ):
#     # send pos/data to workers
#     for i, dat in enumerate(data_list):
#         inQ.put((i, dat))

#     # process results
#     for i in range(len(data_list)):
#         ret = outQ.get()
#         pos = ret[0]
#         dat = ret[1]
#         data_list[pos] = ret

        
# def main(samp, batch_size=10):
#     # initialize things
#     n_workers = mp.cpu_count()

#     inQ = mp.Queue()
#     outQ = mp.Queue()

#     # instantiate workers
#     workers = [mp.Process(target=consumer, args=(inQ,outQ))
#                for i in range(n_workers)]

#     # start the workers
#     for w in workers:
#         w.start()

#     dataset = []
#     data_list = []
    
#     for i, row in enumerate(samp.iterrows()):
#         if i and i % batch_size == 0:
            
#             # gather some data
#             data_list = [row for row in samp.iterrows()]

#             # lets process the data a few times
#             process_data(data_list, inQ, outQ)

#             # tell all workers, no more data (one msg for each)
#             for i in range(n_workers):
#                 inQ.put(None)

#             # join on the workers
#             for w in workers:
#                 w.join()

#             dataset.extend(data_list)
#             data_list = []

#         data_list.append(row)


#     # lets process the data a few times
#     process_data(data_list, inQ, outQ)

#     # tell all workers, no more data (one msg for each)
#     for i in range(n_workers):
#         inQ.put(None)
        
#     # join on the workers
#     for w in workers:
#         w.join()

#     dataset.extend(data_list)
# # #     # print out final results  (i*16)
# #     for i,dat in enumerate(data_list):
# #         print i, dat

#     return dataset


In [132]:
%%time
samp = train_df
ds = main(samp=samp)
dds = pd.concat([samp, ds], axis=1)
dds.to_hdf('kaggle-quora', 'train_df')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000


Process Process-219:
    self.run()
Process Process-217:
Traceback (most recent call last):
Process Process-220:
Process Process-218:
Traceback (most recent call last):
  File "/home/avsolatorio/Canopy/appdata/canopy-1.4.1.1975.rh5-x86_64/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
    self.run()
  File "/home/avsolatorio/Canopy/appdata/canopy-1.4.1.1975.rh5-x86_64/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/avsolatorio/Canopy/appdata/canopy-1.4.1.1975.rh5-x86_64/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/avsolatorio/Canopy/appdata/canopy-1.4.1.1975.rh5-x86_64/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/avsolatorio/Canopy/appdata/canopy-1.4.1.1975.rh5-x86_64/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
  File "/home/avsolatorio/Canopy/appdata/canopy-1.4.1

KeyboardInterrupt: 

In [122]:
dds

,id,qid1,qid2,question1,question2,is_duplicate,cv,length_diff,word_num_diff,wv
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.944067,9.0,2.0,0.953420
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.823531,37.0,5.0,0.561796
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.398727,14.0,4.0,0.082986
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.011027,15.0,2.0,0.000000
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.220276,37.0,6.0,0.077710
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,0.489333,4.0,0.0,0.340561
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0,0.024651,43.0,7.0,0.000000
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,0.623099,11.0,2.0,0.289969
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0,0.256409,2.0,0.0,0.904062
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0,0.680133,11.0,0.0,0.225348


In [ ]:
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, roc_auc_score

char_lm_model = LogisticRegression(C=100)
word_lm_model = LogisticRegression(C=1)
length_diff_lm_model = LogisticRegression(C=1)
word_num_diff_lm_model = LogisticRegression(C=1)
rf_model = RandomForestClassifier(n_estimators=100, min_samples_leaf=2, min_samples_split=3, n_jobs=-1)

def fit_models(X, y):
    rf_model.fit(X_train, y_train)
    char_lm_model.fit(X_train.cv.values.reshape(-1, 1), y_train)
    word_lm_model.fit(X_train.wv.values.reshape(-1, 1), y_train)
    length_diff_lm_model.fit(X_train.length_diff.values.reshape(-1, 1), y_train)
    word_num_diff_lm_model.fit(X_train.word_num_diff.values.reshape(-1, 1), y_train)

def predict(X):
    weights = dict(zip(X.columns, rf_model.feature_importances_))
    char_pred = char_lm_model.predict_proba(X.cv.values.reshape(-1, 1))[:, 1] # * weights['cv']
    word_pred = word_lm_model.predict_proba(X.wv.values.reshape(-1, 1))[:, 1] # * weights['wv']
    length_diff_pred = length_diff_lm_model.predict_proba(X.length_diff.values.reshape(-1, 1))[:, 1] # * weights['length_diff']
    word_num_diff_pred = word_num_diff_lm_model.predict_proba(X.word_num_diff.values.reshape(-1, 1))[:, 1] # * weights['word_num_diff']
    rf_pred = rf_model.predict_proba(X)[:, 1]

    return [char_pred, word_pred, length_diff_pred, word_num_diff_pred, rf_pred]

In [ ]:
%%time
fit_models(X_train, y_train)

In [ ]:
print(log_loss(y_test, np.mean(predict(X_test), axis=0)))
print(roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1]))
print(sum((y_test - (1 * (rf_model.predict_proba(X_test)[:, 1] < 0.5))) != 0))
print(list(zip(X_test.columns, rf_model.feature_importances_)))

In [ ]:
from IPython.display import FileLink
test_df = pd.read_csv('../input/test.csv')

scores_data = []
samp = train_df

for row in samp.iterrows():
    if row[0] % 5000 == 0:
        print(row[0])

    scores_data.append(score_row(row))

X = pd.DataFrame(scores_data)
is_duplicate_test = np.mean(predict(X), axis=0)

log_max_mem_usage()